#  Tensorflow training 
In this tutorial, you will train a mnist model in TensorFlow.

## Prerequisites
* Understand the [architecture and terms](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture) introduced by Azure Machine Learning (AML)
* Go through the prerequisites in the [Azure Machine Learning documentation](https://docs.microsoft.com/azure/machine-learning/service/tutorial-train-models-with-aml#prerequisites):
    * install the AML SDK
    * create a workspace and its configuration file (`config.json`)
[//]: # * Review the [tutorial](../train-hyperparameter-tune-deploy-with-tensorflow/train-hyperparameter-tune-deploy-with-tensorflow.ipynb) on single-node TensorFlow training using the SDK

In [ ]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

## Initialize workspace
Initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace you created in the Prerequisites step. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`.

In [ ]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

## Specify existing Kubernetes Compute

In [ ]:
from azureml.core.compute import ComputeTarget, KubernetesCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your Kubernetes compute
compute_name = 'gpucluster-1x'
compute_target = ComputeTarget(workspace=ws, name=compute_name)

In [ ]:
compute_target

## Create a Dataset for Files
A Dataset can reference single or multiple files in your datastores or public urls. The files can be of any format. Dataset provides you with the ability to download or mount the files to your compute. By creating a dataset, you create a reference to the data source location. If you applied any subsetting transformations to the dataset, they will be stored in the dataset as well. The data remains in its existing location, so no extra storage cost is incurred. [Learn More](https://aka.ms/azureml/howto/createdatasets)

In [ ]:
#initialize file dataset 
from azureml.core.dataset import Dataset
web_paths = ['http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz',
             'http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz',
             'http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz',
             'http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz'
            ]
dataset = Dataset.File.from_files(path = web_paths)

you may want to register datasets using the register() method to your workspace so they can be shared with others, reused across various experiments, and referred to by name in your training script.

In [ ]:
#register dataset to workspace
dataset = dataset.register(workspace = ws,
                           name = 'mnist dataset for Arc',
                           description='training and test dataset',
                           create_new_version=True)

In [ ]:
# list the files referenced by dataset
dataset

## Train model on the Kubernetes compute

### Create a project directory
Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script, and any additional files your training script depends on.

In [ ]:
import os
script_folder = './tf-resume-training'
os.makedirs(script_folder, exist_ok=True)

Copy the training script `tf_mnist_with_checkpoint.py` into this project directory.

In [ ]:
import shutil

# the training logic is in the tf_mnist_with_checkpoint.py file.
shutil.copy('./tf_mnist_with_checkpoint.py', script_folder)

shutil.copy('./utils.py', script_folder)

### Create an experiment
Create an [Experiment](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#experiment) to track all the runs in your workspace for this distributed TensorFlow tutorial. 

In [ ]:
from azureml.core import Experiment

experiment_name = 'akse-arc-tf-training1'
experiment = Experiment(ws, name=experiment_name)

### Create ScriptRun

In [ ]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

env = Environment("env-tf")
cd = CondaDependencies.create(pip_packages=['azureml-dataset-runtime[pandas,fuse]', 'azureml-defaults','tensorflow==1.13.1','horovod==0.16.1'])
env.docker.base_image='mcr.microsoft.com/azureml/openmpi4.1.0-cuda11.0.3-cudnn8-ubuntu18.04'
env.python.conda_dependencies = cd

# Register environment to re-use later
env.register(workspace = ws)

from azureml.core import ScriptRunConfig
args=['--data-folder', dataset.as_named_input('mnist').as_mount()]
src = ScriptRunConfig(source_directory=script_folder,
                      script='tf_mnist_with_checkpoint.py',
                      compute_target=compute_target,
                      environment=env,
                      arguments=args)                     

In the above code, we passed our training data reference `ds_data` to our script's `--data-folder` argument. This will 1) mount our datastore on the remote compute and 2) provide the path to the data zip file on our datastore.

### Submit job
### Run your experiment . Note that this call is asynchronous.

In [ ]:
run = experiment.submit(src)
print(run)

In [ ]:
run

### Monitor your run
You can monitor the progress of the run with a Jupyter widget. Like the run submission, the widget is asynchronous and provides live updates every 10-15 seconds until the job completes.